In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Quick planning

- UX design: Design thinking approach
- Modeling language: UML
- Programming type: Object-Oriented Programming
- Environments: pandas, scikit-learn, pmdarima
- Computer-science field: Machine Learning
- Knowledge field: trading - International Finance Management

# Quick Idea

- Get dataset
- Get trading models
- Get a trading model tuner
- Get a trading model optimizer
- Get a returns optimizer
- Get a submission file

# Request of missing packages
Further information on the request guide is available at [Missing Packages](http://https://github.com/Kaggle/docker-python/wiki/Missing-Packages)

Missing packages
- [pmdarima](http://https://pypi.org/project/pmdarima/)

In [ ]:
#!pip install pmdarima

# System Mode

"""
Turn to code when testing
"""
try:
    print("Set the system mode (1) Test mode (2) Commissioning mode ")
    systemModeOption = int(input())

except StdinNotImplementedError or NameError:
    print("System runs without support of input requests due to Kaggle competition rules.")
    systemModeOption = 2

if(systemModeOption == 1): systemMode = 'test'
if(systemModeOption == 2): systemMode = 'commissioning'

In [ ]:
print("System runs without support of input requests due to Kaggle competition rules.")
systemModeOption = 2
if(systemModeOption == 1): systemMode = 'test'
if(systemModeOption == 2): systemMode = 'commissioning'

# Phase 0 | Get raw dataset and customization

In [ ]:
def getCustomization():
    
    print("=========================================")
    print("Project : Market prediction")
    print("=========================================")
    print("Customization")
    print("=========================================")
    
    # Input testing type
    print("\nRegression method for a linear model")
    preference0a = input("(1) Ordinary Least Squares (Cross-validation 10 folds) (2) Bayesian Ridge (Cross-validation 10 folds) (3) Bayesian Ridge: " )
    preference0a = int(preference0a)
    
    if(preference0a == 1): # Ordinary Least Squares
        preference0b = 'NA'
        # Input sample size | train
        preference1 = input("Train dataset | Input instances size (0 - 1,048,575 ): ")
        preference1 = int(preference1)
        preference2 = 'NA'

    
    if(preference0a == 2): # Bayesian Ridge (Cross validaation | 10 folds)
        preference0b = 'NA'
        # Input sample size | train
        preference1 = input("Train dataset | Input instances size (0 - 1,048,575 ): ")
        preference1 = int(preference1)
        preference2 = 'NA'
    
    if(preference0a == 3): # Bayesian Ridge
        preference0b = 'NA'
        # Input sample size | train
        preference1 = input("Train dataset | Input instances size (0 - 1,048,575 ): ")
        preference1 = int(preference1)
        preference2 = 'NA'
        

    preference0 = [preference0a] + [preference0b]
    
    # Input prefered features | train and test
    
    preference3Previous = input("\nMarket Features | Input (1) 3 features (2) 129 features: ")
    preference3Previous = int(preference3Previous)
    
    if(preference3Previous == 1):
        print("\nMarket Features | Input 3 numbers from 0 to 129 ")
        preference3a = input("   Number 1: ")
        preference3a = 'feature_' + str(preference3a)
        
        preference3b = input("   Number 2: ")
        preference3b = 'feature_' + str(preference3b)
        
        preference3c = input("   Number 3: ")
        preference3c = 'feature_' + str(preference3c)
        
        preference3 = [preference3a] + [preference3b] + [preference3c]
    
    else:
        featuresToInclude = []
        
        for i in range(0,130):
            itemToInclude = 'feature_' + str(i)
            featuresToInclude = featuresToInclude + [itemToInclude]
        
        preference3 = featuresToInclude
    
    # Input return value policy to value actions
    ## (1) Model prediction correspondance: 1-point action value when an actual return corresponds to a predicted return and it is within the confidence interval.
    ## (2) Maximax correspondance: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return.
    ## (3) Maximax correspondance biased by tag-confidence perception: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return. Then, the action value has to be multiplied by the confidence probability. Finally, the result must be rounded to the nearest decision-making boundary, i.e., 0 or 1.
    
    print("\n_________________________________________")
    print("\nReturn value policy ")
    print("\n(1) Prediction Model correspondance: Assign 1-point action value when an actual return corresponds to a predicted return and it is within the confidence interval.")
    print("\n(2) Maximax correspondance: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return.")
    print("\n(3) Maximax correspondance biased by tag-confidence perception: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return. Then, the action value has to be multiplied by the confidence probability. Finally, the result must be rounded to the nearest decision-making boundary, i.e., 0 or 1.")
    preference4 = input("\n   Option: ")
    preference4 = int(preference4)
    
    # Input risk exposure to loss
    print("\n_________________________________________")
    print("\nRisk exposure to loss")
    print("\nScenarios | (1) 25% (2) 50% (3) 75% ")
    preference5 = input("\n   Option: ")
    preference5 = int(preference5)
    
    # Input investing proficiency
    # Investing proficiency: (No proficiency) 0.25 - 0.5 - 0.75 - 0.99 (proficiency)
    print("\n_________________________________________")
    print("\nInvesting proficiency")
    print("\nScenarios | (1) 25%-beginner  (2) 50% (3) 75% (4) 99%-expert")
    preference6 = input("\n   Option: ")
    preference6 = int(preference6)
    
    # Build customization vector
    customizationVector = [preference0, preference1, preference2, preference3, preference4, preference5, preference6]
    
    return customizationVector


def getRawDataset(customizationVector):
    """ Function getRawDataset get Kaggle dataset: train, test, submission """
    import pandas as pd
    import numpy as np
    
    # Get datasets

    if(customizationVector[0][0] in [1,2,3]): # Bayesian Ridge

        # Train | Set columns and rows to be used
        usecolsTrain = ['date','resp','resp_1','resp_2','resp_3','resp_4'] + customizationVector[3] + ['weight','ts_id']
        nrowsTrain = customizationVector[1]

        # Train | Raw dataset
        filename = 'train.csv'
        path = '/kaggle/input/jane-street-market-prediction/' #path = 'E:/Kaggle_MarketData/1-Data/'
        rawDataset = pd.read_csv(path+filename, nrows=nrowsTrain, usecols=usecolsTrain)
        labels =  list(rawDataset.index)
        
        # Train
        rawDatasetTrain = rawDataset.drop(labels[nrowsTrain:])
        
        # Test | Set columns and rows to be used
        usecolsTest = ['date'] + customizationVector[3] + ['weight','ts_id']
        nrowsTest = customizationVector[1]

        # Test | Raw dataset
        filename = 'example_test.csv'
        path = '/kaggle/input/jane-street-market-prediction/' #path = 'E:/Kaggle_MarketData/1-Data/'
        rawDataset_test = pd.read_csv(path+filename, nrows=nrowsTest, usecols=usecolsTest)
        labels =  list(rawDataset_test.index)
        
        # Test
        rawDatasetTest = rawDataset_test.drop(labels[nrowsTest:])
        

        
    # Submission file
    filename = 'example_sample_submission.csv'
    path = '/kaggle/input/jane-street-market-prediction/' #path = 'E:/Kaggle_MarketData/1-Data/'
    rawDatasetSubmission = pd.read_csv(path+filename)
    
    # Feature file
    filename = 'features.csv'
    path = '/kaggle/input/jane-street-market-prediction/' #path = 'E:/Kaggle_MarketData/1-Data/'
    rawFeatureDataset = pd.read_csv(path+filename)
    
    return rawDatasetTrain, rawDatasetTest, rawDatasetSubmission, rawFeatureDataset


def f(x, noise_amount):
    """ Support function to predict """
    import numpy as np
    
    y = np.sqrt(x) * np.sin(x)
    noise = np.random.normal(0, 1, len(x))
    
    return y + noise_amount * noise

# Phase 1 | Object Building - Object 1: Linear Model Project

In [ ]:
class LinearModelProject(object):
    """
    LinearModelProject is an Linear Model solution that returns a Kaggle submission. 
    
    """
    
    def __init__(self):
        """ Create empty DataFrame """
        import numpy as np
        import pandas as pd
        import matplotlib.pyplot as plt
        
        # Empty objects
        dictionaryEmpty = {}
        DataFrameEmpty = pd.DataFrame(data=dictionaryEmpty)
        
        # Default DataFrames
        """ getDatasetDefault loads Arima dataset in the view of Smith et al. (2019)"""
        #self.df = load_msft()
        self.dfc = DataFrameEmpty
        
        # DataFrames | datasets (raw)
        self.datasetTrain = DataFrameEmpty
        self.datasetTest = DataFrameEmpty
        
        # DataFrames | datasets (conditioned)
        self.train = DataFrameEmpty
        self.test = DataFrameEmpty

    
    def insert(self,rawDatasetTrain,rawDatasetTest,train,test,customizationVector):
        """ Assumes rawDtasetTrain is the Kaggle-train dataset """
        """ Assumes rawDtasetTest is the Kaggle-test dataset """
        
        # Dataset (raw)
        self.datasetTrain = rawDatasetTrain
        self.datasetTest = rawDatasetTest
        
        # Dataset (conditioned)
        self.train = train
        self.test = test
        
        # Customization vector
        self.customizationVector = customizationVector
    
    def getDatasets(self):
        """ Get inserted datasets """
        
        return self.datasetTrain, self.datasetTest, self.train, self.test

    
    def datasetConditioning(self, dataset):
        """ Conditioning 1: Set date typification in ISO 8601"""
        
        # Libraries
        from datetime import date
        import pandas as pd
        
        self.dfc = dataset

        
        # Typing 'date'
        self.dfc.astype({'date': 'str'})
        
        # Conditioning
        datasetRange = len(self.dfc)
        isoDateList = []
        #startNumber = self.dfc['date'][0]
        startNumber = self.dfc.index[0]
        
        for i in range(startNumber,datasetRange):
            
            # State | Reference date -> year = 2000          
            dayNumber = int(self.dfc.at[i,'date'])
            timesToSubstract_month = dayNumber//30
            day = dayNumber - (timesToSubstract_month * 30)
            if (day == 0): day = 1
            
            monthNumber = timesToSubstract_month
            timesToSubstract_year = monthNumber//12
            month = monthNumber - (timesToSubstract_year * 12)
            if (month == 0): month = 1
            
            year = timesToSubstract_year + 2000

            # Date list maker
            dateToInclude = date(year, month, day).isoformat()
            isoDateList = isoDateList + [dateToInclude]

        # Insert 'date' with ISO 8601 values    
        
        self.dfc = self.dfc.drop(columns='date')
        isoDateList = pd.Series(isoDateList)
        self.dfc.insert(column='date',value=isoDateList,loc=0)
        
        
        return self.dfc
        
    
    def getLinearModel(self,train,test,customizationVector):
        """
        getLinearModel_BayesianRidgeRegression return a linear model solution intended for regression using Bayesian Ridge Regression
        Code adapted from e scikit-learn example available at https://scikit-learn.org/stable/auto_examples/linear_model/plot_bayesian_ridge.html#sphx-glr-auto-examples-linear-model-plot-bayesian-ridge-py
        """
        import numpy as np
        import matplotlib.pyplot as plt
        from scipy import stats
        import pandas as pd
        
        from sklearn.linear_model import BayesianRidge, LinearRegression
        from sklearn.model_selection import cross_val_predict
        
        # Set input DataFrames | y -> resp
        X = train.drop(columns = ['date','resp'])       
        X = X.fillna(0)
        #X = X.to_numpy()
        
        yDictionary = {}
        yDictionary['resp'] = train.resp
        y = pd.DataFrame(data=yDictionary)
        y = y.fillna(0)
        #y = y.to_numpy()
        
        if (customizationVector[0][0] == 1): # Ordinary Least Squares
            # Fit the model
            clf = LinearRegression()
            clf.fit(X, y)
            
            # Fit the model
            clf = BayesianRidge(compute_score=True)
            clf.fit(X, y)
       
            # Get predictions
            # cross_val_predict returns an array of the same size as `y` where each entry
            # is a prediction obtained by cross validation:
            y_pred = cross_val_predict(clf, X, y, cv=10)
            y_pred_std = []
            for i in range(0,len(y_pred)):y_pred_std = y_pred_std + [np.nan]


        if (customizationVector[0][0] == 2): # Bayesian Ridge Regression
            # Fit the model
            clf = BayesianRidge(compute_score=True)
            clf.fit(X, y)
        
            # Get predictions
            y_pred, y_pred_std = clf.predict(X, return_std=True)        
        
        if (customizationVector[0][0] == 3): # Bayesian Ridge Regression | Cross-validation (10 folds)
            # Fit the model
            clf = BayesianRidge(compute_score=True)
            clf.fit(X, y)
       
            # Get predictions
            # cross_val_predict returns an array of the same size as `y` where each entry
            # is a prediction obtained by cross validation:
            y_pred = cross_val_predict(clf, X, y, cv=10)
            y_pred_std = []
            for i in range(0,len(y_pred)):y_pred_std = y_pred_std + [np.nan]
            
        # Plotting Predictions
        fig, ax = plt.subplots()
        ax.scatter(y, y_pred, edgecolors=(0, 0, 0))
        ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
        ax.set_xlabel('Measured')
        ax.set_ylabel('Predicted')
        plt.show()
        
        # Create a DataFrame with forecast instances
        forecastsDictionary = {}
        forecastsDictionary['respPredict'] = y_pred
        forecastsDataFrame = pd.DataFrame(data=forecastsDictionary, index=train.index)

        # Create a Dataframe with confidence intervals
        confidence_intervalsDictionary = {}
        confidence_intervalsDictionary['respStd'] = y_pred_std
        confidence_intervalsDataFrame = pd.DataFrame(data=confidence_intervalsDictionary,index=train.index)

        # Include forecast instances into training dataset
        trainPredict = pd.concat([train, forecastsDataFrame], axis=1, join="inner")
        trainPredict = pd.concat([trainPredict, confidence_intervalsDataFrame], axis=1, join="inner")
        
        # https://scikit-learn.org/stable/auto_examples/linear_model/plot_bayesian_ridge.html#sphx-glr-auto-examples-linear-model-plot-bayesian-ridge-py
        # https://scikit-learn.org/stable/modules/linear_model.html#bayesian-regression
        # https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html#sphx-glr-auto-examples-model-selection-plot-cv-predict-py
        
        return trainPredict

# Phase 1 | Object Building - Object 2: Feature Analytics Project

In [ ]:
class featureAnalytics(object):
    """
    featureAnalytics returns a component to biased models.
    """

    def __init__(self):
        """ Create empty DataFrame """
        import numpy as np
        import pandas as pd
        
        # Empty objects
        dictionaryEmpty = {}
        DataFrameEmpty = pd.DataFrame(data=dictionaryEmpty)
        
        # DataFrames | datasets (raw)
        self.featureDataset = DataFrameEmpty
        
        # DataFrames | datasets (conditioned)
        self.confidenceProbabilityDataset = DataFrameEmpty

    
    def insert(self,customizationVector,rawFeatureDataset):
        """ Assumes rawFeatureDataset is the Kaggle-feature dataset """
        
        # Dataset (raw)
        self.featureDataset = rawFeatureDataset
        
        # Customization vector
        self.customizationVector = customizationVector
    
    
    def getDatasets(self):
        """ Get inserted datasets """
        
        return self.featureDataset
    
    
    def confidenceProbabilityProject(self,featureDataset,customizationVector):
        """ Get confidence probabilities to support computations under a return value policy 3 """
        
        from math import exp
        
        # Get feature mixture
        featureMixture = customizationVector[3]
        
        # Get proficiency constant
        if(customizationVector[6] == 1):proficiencyConstant = 0.25
        if(customizationVector[6] == 2):proficiencyConstant = 0.50
        if(customizationVector[6] == 3):proficiencyConstant = 0.75
        if(customizationVector[6] == 4):proficiencyConstant = 0.99
        
        # Get confidence probability list
        
        trueList = []
        
        
        for i in range(0,len(featureDataset.feature)):
                
            if featureDataset.feature[i] in featureMixture:
                trueSet = sum(featureDataset.loc[i][1:] == True) #series
                
                if(trueSet != 0):
                    trueList = trueList + [trueSet]
        
        # Get confidence probability value
        featureManaged = len(featureMixture)
        probability = sum(trueList)/(featureManaged*29)
        
        # Adjust probability using a success factor due to expertise
        ## Investing proficiency: (No proficiency) 0 - 0.25 - 0.5 - 0.75 - 0.99 (proficiency)
        ## featureManagedAdjustment: Tune constant to adjust the factor so that confidenceProbability is the highest one if proficiency is the best one when managing the maximum possible available features
        ## It is assumed that featureManagedAdjustment implies that feature management goes more suitable in the measure in which more features are managed.
        
        featureManagedAdjustment = exp(2.62) 
        factor = proficiencyConstant * featureManaged / featureManagedAdjustment
        confidenceProbability = probability * factor 
        
        return confidenceProbability #confidenceProbability

# Phase 1 | Object Building - Object 3: Returns optimizer

In [ ]:
class returnsOptimizer(object):
    """
    returnsOptimizer builds action instances maximizing returns on the trade (i.e. resp) given a volatility policy.
    returnsOptimizer returns the utility score in the basis of the optimization
    """
    
    def __init__(self):
        """ Create empty DataFrame """
        import numpy as np
        import pandas as pd
        
        # Empty objects
        dictionaryEmpty = {}
        DataFrameEmpty = pd.DataFrame(data=dictionaryEmpty)
        
        # Default DataFrames
        self.dfc = DataFrameEmpty
        
        # DataFrames | training dataset with predictions
        self.train = DataFrameEmpty
        self.trainAction = DataFrameEmpty
        
        # Lists | customizationVector
        self.customizationVector = []

    
    def insert(self,trainPredict,customizationVector,confidenceProbability):
        """ Assumes rawDtasetTrain is the Kaggle-train dataset """
        """ Assumes rawDtasetTest is the Kaggle-test dataset """
        
        # Variables
        self.probability = confidenceProbability
        
        # Dataset (conditioned)
        self.train = trainPredict
        
        # Customization vector
        self.customizationVector = customizationVector


    def actionValuer(self,train,customizationVector,probability):
        """
        Policy 1 - Model prediction correspondance
        Action valuer policy due to Kaggle competition context (Optimizing criterion)
        << Assign 1 to action value if the test return value keeps similarity with
        its corresponding prediction instance. >>
        i.e. test return value keeps within the confidence interval computed for predicting
        Underpinned statement (theory): Outliers, test instances outside the confidence
        interval of the prediction model, disrupt the model so those instances are deemed under loss exposure.
        
        Policy 2 - Maximax correspondance
        << Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return." >>
        Underpinned statement (theory): Selecting just gain instances (i.e. + actual returns and + predicted returns) will reach return maximization.
        
        Policy 3 - Maximax correspondance biased by tag-confidence perception
        <<Assign points as policy 2 but times a confidence probability.>>
        
        Good Practice theory on Jane Street Investment manners: "Trade-decision making using ML models to predict is biased by a tailored milestone detection."
        
        Assumptions:
        (1) In Kaggle competition, the tag assignment is provided as "features.csv".
        (2) The tag assignment refers to a view of the return behaviour in terms of tracing milestones given a set of trading transactions.
        (3) A feature mixture requires a confidence probability to biased model prediction values due to a given tag assignment.
        (4) A tag is deemed an attribute that specifies a feature. 
        (5) A tag-assignment refers to flag a confidence attribute that configures a feature customization for predicting.
        (6) The confidence probability implies a trade decision if it is more than a minimum probability of occurence to trade. In other words, the confidence probability is bounded below to a minimum expected probability of occurrence
        (7) The probability of occurrence to trade implies the minimum affordable probability to avoid a loss risk. Hence, if the confidence probability is smaller than the probabilit of occurrence, it is recommende to avoid trading.
        """
        # Set risk exposure to loss
        if(customizationVector[5] == 1): riskExposure = 0.75
        if(customizationVector[5] == 2): riskExposure = 0.50
        if(customizationVector[5] == 3): riskExposure = 0.25
        
        
        if (customizationVector[4] == 1): # Return Value policy does not apply in this case; it is for ARIMA.
            option = customizationVector[0][0]
            if(option in [1,2,3]):
            
                import pandas as pd
                
                # Value assignment
                actionList = []
                trainRange = len(train)
                for i in range(0,trainRange):
                    
                    itemToValue = train.resp[i]
                    lowerBoundary = train.confInterval[i][0]
                    upperBoundary = train.confInterval[i][1]
                    
                    
                    if(itemToValue >= lowerBoundary and itemToValue <= upperBoundary and probability >= riskExposure):
                        actionList = actionList + [1]
                    else:
                        actionList = actionList + [0]
                
                # Concatenate DataFrames| trainPredict and action
                
                ## Create a DataFrame with forecast instances
                actionDictionary = {}
                actionDictionary['action'] = actionList
                actionDataFrame = pd.DataFrame(data=actionDictionary, index=train.index)
        
                ## Include action instances into training dataset
                trainAction = pd.concat([train, actionDataFrame], axis=1, join="inner")
                self.trainAction = trainAction
        
        if (customizationVector[4] == 2):
            option = customizationVector[0][0]
            if(option in [1,2,3]):
            
                import pandas as pd
                
                # Value assignment
                actionList = []
                trainRange = len(train)
                for i in range(0,trainRange):
                    
                    itemToValue = train.resp[i]
                    
                    if(itemToValue > 0 and probability >= riskExposure):
                        actionList = actionList + [1]
                    else:
                        actionList = actionList + [0]
                
                # Concatenate DataFrames| trainPredict and action
                
                ## Create a DataFrame with forecast instances
                actionDictionary = {}
                actionDictionary['action'] = actionList
                actionDataFrame = pd.DataFrame(data=actionDictionary, index=train.index)
        
                ## Include action instances into training dataset
                trainAction = pd.concat([train, actionDataFrame], axis=1, join="inner")
                self.trainAction = trainAction
                
        if (customizationVector[4] == 3):
            option = customizationVector[0][0]
            if(option in [1,2,3]):
            
                import pandas as pd
                
                actionList = []
                trainRange = len(train)
                for i in range(0,trainRange):
                    
                    itemToValue = train.resp[i]
                    
                    if(itemToValue > 0 and probability >= riskExposure):
                        actionList = actionList + [1]
                    else:
                        actionList = actionList + [0]

                
                # Concatenate DataFrames| trainPredict and action
                
                ## Create a DataFrame with forecast instances
                actionDictionary = {}
                actionDictionary['action'] = actionList
                actionDataFrame = pd.DataFrame(data=actionDictionary, index=train.index)
        
                ## Include action instances into training dataset
                trainAction = pd.concat([train, actionDataFrame], axis=1, join="inner")
                self.trainAction = trainAction
            
        
        return self.trainAction
    
    
    def utilityScore(self,train):
        """
        Returns utility score proposed by Kaggle
        """
        import pandas as pd
        
        # Create pUnitary DataFrame
        pUnitaryDictionary = {}
        pUnitary2Dictionary = {}
        pUnitaryList = []
        pUnitary2List = []
        
        for i in train.index: # Issue -0.0

            weight = train.weight[i]
            resp = train.resp[i]
            action = train.action[i]
            
            pUnitary = weight * resp * action
            pUnitaryList = pUnitaryList + [pUnitary]
            
            pUnitary2 = pUnitary**2 
            pUnitary2List = pUnitary2List + [pUnitary2]
        
        pUnitaryDictionary['pUnitary'] = pUnitaryList
        pUnitaryDataFrame = pd.DataFrame(data=pUnitaryDictionary, index=train.index)
        
        pUnitary2Dictionary['pUnitary2'] = pUnitary2List
        pUnitary2DataFrame = pd.DataFrame(data=pUnitary2Dictionary, index=train.index)
        
        # Concatenate DataFrames | train and pUnitaryDataFrame
        trainPUnitary_transition = pd.concat([train, pUnitaryDataFrame], axis=1, join="inner")
        
        # Concatenate DataFrames | train and pUnitaryDataFrame
        trainPUnitary = pd.concat([trainPUnitary_transition, pUnitary2DataFrame], axis=1, join="inner")
        
        # Grouped by date | trainPUnitary
        grouped = trainPUnitary.groupby(by=['date','pUnitary','pUnitary2']).count()
        
        # Summation lists
        i = 0
        
        dateList = []
        pUnitaryGroupedList = []
        pUnitary2GroupedList = []
        dataFrequencyList = []
        
        for i in range(0,len(grouped.index)):
            
            pUnitary = grouped.index[i][1]
            pUnitary2 = grouped.index[i][2]
            
            if(i == 0):
            
                date = grouped.index[i][0]
                dateList = dateList + [date]
                
                pUnitary = grouped.index[i][1]
                pUnitaryGroupedList = pUnitaryGroupedList + [pUnitary]
                
                pUnitary2 = grouped.index[i][2]
                pUnitary2GroupedList = pUnitary2GroupedList + [pUnitary2]
                
                datum = 1
                dataFrequencyList = dataFrequencyList + [datum]
            
            else:
                
                if(grouped.index[i-1][0] == grouped.index[i][0]):
                
                    instance = len(pUnitaryGroupedList)-1
                    pUnitaryGroupedList[instance] = pUnitaryGroupedList[instance] + pUnitary
                    pUnitary2GroupedList[instance] = pUnitary2GroupedList[instance] + pUnitary2
                    dataFrequencyList[instance] = dataFrequencyList[instance] + 1
                    
                else:
                    
                    date = grouped.index[i][0]
                    dateList = dateList + [date]
                
                    pUnitary = grouped.index[i][1]
                    pUnitaryGroupedList = pUnitaryGroupedList + [pUnitary]
                    
                    pUnitary2 = grouped.index[i][1]
                    pUnitary2GroupedList = pUnitary2GroupedList + [pUnitary2]
                    
                    datum = 1
                    dataFrequencyList = dataFrequencyList + [datum]
        
        # Create DataFrame | utilityScoreDataFrame
        utilityScoreDictionary = {}
        utilityScoreDictionary['date'] = dateList
        utilityScoreDictionary['p'] = pUnitaryGroupedList
        utilityScoreDictionary['p2'] = pUnitary2GroupedList
        utilityScoreDictionary['frequency'] = dataFrequencyList
        
        utilityScoreDataFrame = pd.DataFrame(data=utilityScoreDictionary)
        
        # Compute utilityScore
        ## sum p and sum P2
        sum_p = sum(utilityScoreDataFrame.p)
        sum_p2 = sum(utilityScoreDataFrame.p2)
        ## Number of unique dates
        uniqueDates = len(utilityScoreDataFrame.frequency)
        ## t
        t = (sum_p / sum_p2**0.5) * (250/uniqueDates)**0.5
        ## utilityScore
        utilityScore = min(max(t,0),6) * sum_p
        
        print("=========================================")
        print("Utility Score | Data")
        print("=========================================")
        print("Summation p: ",sum_p)
        print("t: ",t)
        print("u: ",utilityScore)
        print("=========================================")        
        
        return trainPUnitary,utilityScoreDataFrame,utilityScore


    def submission(self,train_raw,trainAction,customizationVector):
        """
        submission delivery
        """
        import pandas as pd
        train = trainAction.copy()
        train['date'] = train_raw['date'].astype('int32')
        
        from tqdm import tqdm     
        import janestreet
        janestreet.make_env.__called__ = False
        env = janestreet.make_env() # initialize the environment
        iter_test = env.iter_test() # an iterator which loops over the test set
        
        for (test_df, sample_prediction_df) in tqdm(iter_test):
            #sample_prediction_df.action = train.action.astype('int32')
            sample_prediction_df["action"] = train["action"].astype('int32')
            env.predict(sample_prediction_df)

# Phase 2 | Test mode

In [ ]:
if systemMode == 'test':
    
    # Get customization
    customizationVector = getCustomization()
    
    # Get raw dataset
    rawDatasetTrain, rawDatasetTest, rawDatasetSubmission, rawFeatureDataset = getRawDataset(customizationVector)
    
    
    """ Build object 1 | class Linear Model Project """
    
    # Buil object
    LinearModelProject1 = LinearModelProject()
    
    # Call method insert
    train = [] # Conditioning in progress
    test = [] # Conditioning in progress
    LinearModelProject1.insert(rawDatasetTrain,rawDatasetTest,train,test,customizationVector)
    
    # Call method getDatasets
    train_raw, test_raw, train, test = LinearModelProject1.getDatasets()
    
    # Call method datasetConditioning
    train = LinearModelProject1.datasetConditioning(train_raw)
    test = LinearModelProject1.datasetConditioning(test_raw)
    
    # Call method insert
    LinearModelProject1.insert(rawDatasetTrain,rawDatasetTest,train,test,customizationVector)
        
    # Call method getLinearModel(self,dataset,datasetType,customizationVector)
    ## train dataset
    trainPredict = LinearModelProject1.getLinearModel(train,test,customizationVector)
    
    
    """ Build object 2 | class Feature Analytics Project """
    # Build object
    FeatureAnalyticsProject1 = featureAnalytics()
    
    # Call method insert
    FeatureAnalyticsProject1.insert(customizationVector,rawFeatureDataset)
    
    # Call method getDatasets
    featureDataset = FeatureAnalyticsProject1.getDatasets()
    
    # Call method 
    confidenceProbability = FeatureAnalyticsProject1.confidenceProbabilityProject(featureDataset,customizationVector)
    
    
    """ Build object 3 | returnsOptimizer """
    # Build object
    LinearModelProject1_Submission = returnsOptimizer()
    
    # Call method insert
    LinearModelProject1_Submission.insert(trainPredict,customizationVector,confidenceProbability)
    
    # Call method actionValuer
    trainAction = LinearModelProject1_Submission.actionValuer(trainPredict,customizationVector,confidenceProbability)
    
    # Call method utilityScore
    trainPUnitary,utilityScoreDataFrame,utilityScore = LinearModelProject1_Submission.utilityScore(trainAction)

# Phase 3 | Commissioning mode

* System mode | Options: (1) Test (2) Commissioning 

      --> Commissioning mode
    

* Regression method for a linear model | Options: (1) Ordinary Least Squares (Cross-validation 10 folds) (2) Bayesian Ridge (Cross-validation 10 folds) (3) Bayesian Ridge

      --> Bayesian Ridge

* Train dataset | Input instances size (0 - 1,048,575 )

      --> 15219

* Return value policy | (1) Prediction Model correspondance: Assign 1-point action value when an actual return corresponds to a predicted return and it is within the confidence interval (2) Maximax correspondance: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return (3) Maximax correspondance biased by tag-confidence perception: Assign 1-point action value when a (+) actual return corresponds to a (+) predicted return. Then, the action value has to be multiplied by the confidence probability. Finally, the result must be rounded to the nearest decision-making boundary, i.e., 0 or 1..

      --> (3)

* Risk exposure to loss | Scenarios | (1) 25% (2) 50% (3) 75% 

      --> (1)

* Investing proficiency | Scenarios | (1) 25%-beginner  (2) 50% (3) 75% (4) 99%-expert

      --> (4)

In [ ]:
%%time

if systemMode == 'commissioning':
    
    # Get customization
    
    customizationVector = [[3, 'NA'],
     15219,
     'NA',
     ['feature_0',
      'feature_1',
      'feature_2',
      'feature_3',
      'feature_4',
      'feature_5',
      'feature_6',
      'feature_7',
      'feature_8',
      'feature_9',
      'feature_10',
      'feature_11',
      'feature_12',
      'feature_13',
      'feature_14',
      'feature_15',
      'feature_16',
      'feature_17',
      'feature_18',
      'feature_19',
      'feature_20',
      'feature_21',
      'feature_22',
      'feature_23',
      'feature_24',
      'feature_25',
      'feature_26',
      'feature_27',
      'feature_28',
      'feature_29',
      'feature_30',
      'feature_31',
      'feature_32',
      'feature_33',
      'feature_34',
      'feature_35',
      'feature_36',
      'feature_37',
      'feature_38',
      'feature_39',
      'feature_40',
      'feature_41',
      'feature_42',
      'feature_43',
      'feature_44',
      'feature_45',
      'feature_46',
      'feature_47',
      'feature_48',
      'feature_49',
      'feature_50',
      'feature_51',
      'feature_52',
      'feature_53',
      'feature_54',
      'feature_55',
      'feature_56',
      'feature_57',
      'feature_58',
      'feature_59',
      'feature_60',
      'feature_61',
      'feature_62',
      'feature_63',
      'feature_64',
      'feature_65',
      'feature_66',
      'feature_67',
      'feature_68',
      'feature_69',
      'feature_70',
      'feature_71',
      'feature_72',
      'feature_73',
      'feature_74',
      'feature_75',
      'feature_76',
      'feature_77',
      'feature_78',
      'feature_79',
      'feature_80',
      'feature_81',
      'feature_82',
      'feature_83',
      'feature_84',
      'feature_85',
      'feature_86',
      'feature_87',
      'feature_88',
      'feature_89',
      'feature_90',
      'feature_91',
      'feature_92',
      'feature_93',
      'feature_94',
      'feature_95',
      'feature_96',
      'feature_97',
      'feature_98',
      'feature_99',
      'feature_100',
      'feature_101',
      'feature_102',
      'feature_103',
      'feature_104',
      'feature_105',
      'feature_106',
      'feature_107',
      'feature_108',
      'feature_109',
      'feature_110',
      'feature_111',
      'feature_112',
      'feature_113',
      'feature_114',
      'feature_115',
      'feature_116',
      'feature_117',
      'feature_118',
      'feature_119',
      'feature_120',
      'feature_121',
      'feature_122',
      'feature_123',
      'feature_124',
      'feature_125',
      'feature_126',
      'feature_127',
      'feature_128',
      'feature_129'],
     3,1,4]
    
    # Get raw datasets
    rawDatasetTrain, rawDatasetTest, rawDatasetSubmission, rawFeatureDataset = getRawDataset(customizationVector)
    
    
    """ Build object 1 | class Linear Model Project """
    
    # Buil object
    LinearModelProject1 = LinearModelProject()
    
    # Call method insert
    train = [] # Conditioning in progress
    test = [] # Conditioning in progress
    LinearModelProject1.insert(rawDatasetTrain,rawDatasetTest,train,test,customizationVector)
    
    # Call method getDatasets
    train_raw, test_raw, train, test = LinearModelProject1.getDatasets()
    
    # Call method datasetConditioning
    train = LinearModelProject1.datasetConditioning(train_raw)
    test = LinearModelProject1.datasetConditioning(test_raw)
    
    # Call method insert
    LinearModelProject1.insert(rawDatasetTrain,rawDatasetTest,train,test,customizationVector)
        
    # Call method getLinearModel(self,dataset,datasetType,customizationVector)
    ## train dataset
    trainPredict = LinearModelProject1.getLinearModel(train,test,customizationVector)
    
    
    """ Build object 2 | class Feature Analytics Project """
    # Build object
    FeatureAnalyticsProject1 = featureAnalytics()
    
    # Call method insert
    FeatureAnalyticsProject1.insert(customizationVector,rawFeatureDataset)
    
    # Call method getDatasets
    featureDataset = FeatureAnalyticsProject1.getDatasets()
    
    # Call method 
    confidenceProbability = FeatureAnalyticsProject1.confidenceProbabilityProject(featureDataset,customizationVector)
    
    
    """ Build object 3 | returnsOptimizer """
    # Build object
    LinearModelProject1_Submission = returnsOptimizer()
    
    # Call method insert
    LinearModelProject1_Submission.insert(trainPredict,customizationVector,confidenceProbability)
    
    # Call method actionValuer
    trainAction = LinearModelProject1_Submission.actionValuer(trainPredict,customizationVector,confidenceProbability)
    
    # Call method utilityScore
    trainPUnitary,utilityScoreDataFrame,utilityScore = LinearModelProject1_Submission.utilityScore(trainAction)    
    
    # Call method submission
    LinearModelProject1_Submission.submission(train_raw,trainAction,customizationVector)

# Bibliography

[0] Design

Larman, G. (2002). Applying UML and Patterns: An introduction to
    Objected-Oriented Analysis and Design and the Unified Process (2nd ed.).
    New Jersey, The United States of America: Prentice Hall.

Soegaard, M. (2018). The Basics of User Experience Design: A UX Design Book by
    the Interaction Design Foundation (1st ed.). Denmark: Interaction Design
    Foundation.


[1] Programming
Guttag, J. (2013). Introduction to Computation and Programming Using Python.
    Massachusetts, The United States of America: The MIT Press.

Lott, S. (2019). Mastering Object-Oriented Python | Build powerful applications
    with reusable code using OOP design patterns and Python 3.7 (2nd ed.).
    Birmingham, United Kingdom: Packt Publishing Ltd.

Smith, Taylor G., et al. pmdarima: ARIMA estimators for Python, 2017-,
    http://www.alkaline-ml.com/pmdarima [Online; accessed 2020-12-18].


[2] Computer Science

Raschka, S., & Mirjalili, V. (2019). Python Machine Learning: Machine Learning
    and Deep Learning with Python, scikit-learn, and TensorFlow 2 (3rd ed.).
    Birmingham, United Kingdom: Packt Publishing Ltd.


[3] Knowledge field

Baekert, G., & Hodrick, R. (2012). International Financial Management (2nd ed.).
    New York, The United States of America: Pearson Education, Inc.

Yan, Y. (2017). Python for Finance | Apply powerful finance models and
    quantitative analysis with Python (2nd ed.). Birmingham, United Kingdom:
        Packt Publishing Ltd.

Lagplot. (2016). Available at https://www.statisticshowto.com/lag-plot/
        

[4] Technical Notes

Issue: NumPy 1.19.4 to NumPy 1.19.3 because of RuntimeError
Note: https://tinyurl.com/y3dm3h86 (see (1) hessler.evan
03 Nov a 03: 28; (2) Kevin Sheppard 03 Nov a 08: 04)

Issue: What everyone is looking for: API Workarounds 
Note: https://www.kaggle.com/c/jane-street-market-prediction/discussion/200024